In [37]:
import re

path = "./datapoints.txt"  

cleaned_data = []


with open(path, "r") as f_read:
    for line in f_read:
        line = line.strip() 
        line = re.sub(r"[()]", "", line)
       
    #   This divides each line into parts to isolate each variable. It also cleans up each part then adds them to an array of parts.
        parts = [part.strip() for part in line.split(",")]
        
        # This checks if each line has 3 parts, and if it does, sorts them into the correct part-type.
        if len(parts) == 3:
            width, height, label = parts
            # The code below adds the parts to the cleaned_data list as as dictionary with floats or ints depending on what fits best. 
            # It also labels each of the parts in the dictionary as key/value pairs so that we can more easily reference them later on.
            try:
                cleaned_data.append({
                    "width": float(width),  
                    "height": float(height),  
                    "label": int(label) 
                })
            except ValueError:
                # If there is anything wrong with any of the values we attempt to catch it here and stop it from being added to the cleaned data
                print(f"This line has one or more values that are faulty: {line}")  

# Adding this comment as a reminder to myself - 
# to refer to a specific value in a dictionary you can type "dictionary['key']" like so to find it  
for data_point in cleaned_data:
    print(f"Width: {data_point['width']}, Height: {data_point['height']}, Label: {data_point['label']}")

Width: 21.959384499160468, Height: 31.23956701424158, Label: 0
Width: 23.63591632187622, Height: 36.46821490673444, Label: 1
Width: 17.714056417303343, Height: 31.44170391314962, Label: 0
Width: 21.78020031891186, Height: 33.213397759738264, Label: 1
Width: 25.20630552859312, Height: 36.558816149350534, Label: 1
Width: 19.332572350434354, Height: 32.25325633655492, Label: 0
Width: 24.73645685241186, Height: 35.33291181124776, Label: 1
Width: 23.79257560586339, Height: 38.10372825362463, Label: 1
Width: 24.557612968127465, Height: 36.73144402805611, Label: 1
Width: 20.191281253428173, Height: 35.06966921830237, Label: 0
Width: 25.813562951888365, Height: 35.561029988644336, Label: 1
Width: 24.923378667802954, Height: 34.463907946680294, Label: 1
Width: 25.311244044578427, Height: 34.117212558131975, Label: 1
Width: 22.819091361866796, Height: 34.25516433025548, Label: 1
Width: 19.639358214988224, Height: 34.56117030001663, Label: 0
Width: 18.341233265627693, Height: 31.399261188293124, 